<a href="https://colab.research.google.com/github/slickHnaa/Career_Accelerator_LP3-Classifcation/blob/main/01_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

# **Questions**

Do senior citizens have a higher churn rate than others?

Do customers with dependents have higher church rates?

Does age and gender contribute to the churn rate?

Is there a relationship between tunure and churn rate?

Does the contract term affect the churn rate?

Does number of services signed on to affect churn rate?

# **Hypothesis**

The contract term affects attrition


**Null Hypothesis**

The contract tern does not affect attrition 


**Alternate hypothesis**

The contract term affects attrition

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

#relevant ML libraries
from imblearn.over_sampling import ADASYN
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,StratifiedKFold,RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score
import sklearn.metrics as metrics
from sklearn.neighbors import LocalOutlierFactor



#ML models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier

warnings.filterwarnings('ignore')

In [ ]:
c_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LP3/Telco-Customer-Churn.csv')
c_data.head()

#  Exploratory Data Analysis

## Data Cleaning

In [ ]:
c_data.shape

In [ ]:
c_data.info()

The total charges columns has a data type of an object instead of a float. This must be converted.

In [ ]:
c_data['TotalCharges'] = pd.to_numeric(c_data['TotalCharges'], errors='coerce')

In [ ]:
c_data.dtypes

In [ ]:
c_data.isnull().sum()

There are 11 missing values in the Total Charge column which is very insignificant compared with the total number of entries. Therefore these missing values will be dropped

In [ ]:
# Removing missing values
c_data.dropna(inplace=True)

In [ ]:
c_data.duplicated().sum()

There is no duplicated elements in the dataset

In [ ]:
c_data.shape

The ID column as no use and will be removed

In [ ]:
#removing ID column
c_data = c_data.iloc[:,1:]

In [ ]:
for column in c_data.columns:
  print(column, ':', c_data[column].nunique())


In [ ]:
LE = preprocessing.LabelEncoder()
c_data['Churn'] = LE.fit_transform(c_data['Churn'])
c_data.head()

# Univariate and Bivariate Analysis

## Univariate Analysis

In [ ]:
c_data.describe().T

## **What is the distribution of Billing?**
Billing include the following headings
- Tenure
- Contract
- Paperles Billing
- Payment Method
- Monthly Charge
- Total Charge

#### Monthly Charges



In [ ]:
sns.histplot(x = 'MonthlyCharges', data = c_data).set(title = 'distribution of Monthly Charges')

#### Most customers are charged \$20. Generally, the distribution of monthly charges is skewed to the right. Apart from \$20, most customers are charged between \$40 to \$110.

#### Tenure

In [ ]:
sns.histplot(x = 'tenure', data = c_data).set(title = 'Tunure Across Board')

A lot of new customers sign up for the services and stay generally for 74 months.

#### Paperless Billing

In [ ]:
ax = sns.countplot(x='PaperlessBilling',data=c_data)
vc = c_data['PaperlessBilling'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Count of Customers Paperless Billing')
plt.show()

#### Payment Method

In [ ]:
ax = sns.countplot(x='PaymentMethod',data=c_data)
vc = c_data['PaymentMethod'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Count of Customers  Payment Method')
plt.show()

In [ ]:
ax = sns.countplot(x='PaymentMethod',data=c_data)
vc = c_data['PaymentMethod'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Count of Customers  Payment Method')
plt.show()

#### Distribution of Monthly Charges

In [ ]:
sns.histplot(x = 'MonthlyCharges', data = c_data).set(title = 'Distribution of Monthly Charges')

In [ ]:
sns.histplot(x = 'TotalCharges', data = c_data).set(title = 'Distribution of Total Charges')


#### Count of Customers  Contract

In [ ]:
ax = sns.countplot(x='Contract',data=c_data)
vc = c_data['Contract'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Count of Customers  Contract')
plt.show()

### Count of Customer who Churn or not

In [ ]:
ax = sns.countplot(x='Churn',data=c_data, palette='rainbow')
vc = c_data['Churn'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Count of Customer who Churn or not')
plt.show()

5174 customers did not churn whiles 1869 churn. This makes target variable inblances

### Exploration Into Customer

#### Count of Gender

In [ ]:
ax = sns.countplot(x='gender',data=c_data)
vc = c_data['gender'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Count of Gender')
plt.show()

The dataset contains 3488 females and 3555 males

#### Count of Customers with Partner

In [ ]:
ax = sns.countplot(x='Partner',data=c_data)
vc = c_data['Partner'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Count of Customers with Partner')
plt.show()

The difference between customers with partners and those without partner is 243 which is approximately 3.45% of all customers

#### Count of Customers with Dependents

In [ ]:
ax = sns.countplot(x='Dependents',data=c_data)
vc = c_data['Dependents'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Count of Customers with Dependents')
plt.show()

About 4933 customer do not have dependents whiles 2110 have dependents 

#### Count of Senior Citizen


In [ ]:
ax = sns.countplot(x='SeniorCitizen', data=c_data)
vc = c_data['SeniorCitizen'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Count of Senior Citizen')
plt.show()

There are 5901 non senior citizen and 1142 are senior citizen.

## Bivariate Analysis

In [ ]:
#Term = pd.crosstab(LPcat['Term'],LPcat['Loan Status'])
#Term.div(Term.sum(1).astype(float), axis = 0).plot(kind = 'bar',stacked = True, figsize = (6,6))
#plt.show()

### Gender vs Tenure vs MonthlyCharges

In [ ]:
sns.violinplot(x='gender',y='tenure',data=c_data,palette='rainbow').set(title = 'distribution of Monthly Charges')


In [ ]:
sns.boxplot(x='gender',y='MonthlyCharges',data=c_data).set(title = 'distribution of Monthly Charges')

### Dependents vs Tenure vs MonthlyCharges

In [ ]:
sns.boxplot(x='Dependents',y='tenure',data=c_data).set(title = 'distribution of Monthly Charges')


In [ ]:
sns.violinplot(x='Dependents',y='MonthlyCharges',data=c_data, palette='rainbow').set(title = 'distribution of Monthly Charges')


### Churn vs tenure vs MonthlyCharges

In [ ]:
sns.boxplot(x='Churn',y='tenure', data=c_data).set(title = 'distribution of Monthly Charges')


In [ ]:
sns.violinplot(x='Churn',y='MonthlyCharges', data=c_data, palette='rainbow').set(title = 'distribution of Monthly Charges')


### Pairgrid

In [ ]:
pair_grid = sns.pairplot(c_data)

## Deep dive into relationship with Churn 

What is the relationship between Churn and all the services offerd by vodafone? 
 
The services offered by Vodafone include:
- PhoneService         
- MultipleLines        
- InternetService      
- OnlineSecurity       
- OnlineBackup         
- DeviceProtection     
- TechSupport          
- StreamingTV          
- StreamingMovies 

#### Relationship between Phone Service and Churn


In [ ]:
sns.set_style('darkgrid')     
sns.countplot(x='PhoneService', hue='Churn', data=c_data, palette='Blues')
vc = c_data['PhoneService'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between Phone Service and Churn')
plt.show()


#### Relationship between Multiple Lines and Churn

In [ ]:
sns.set_style('darkgrid')     
sns.countplot(x='MultipleLines', hue='Churn', data=c_data, palette='Blues')
vc = c_data['MultipleLines'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between Multiple Lines and Churn')
plt.show()

#### Relationship between Online Backup and Churn   


In [ ]:
sns.set_style('darkgrid')     
sns.countplot(x='OnlineBackup', hue='Churn', data=c_data, palette='Blues')
vc = c_data['OnlineBackup'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between Online Backup and Churn')
plt.show()

#### Relationship between Online Security and Churn       


In [ ]:
sns.set_style('darkgrid')     
sns.countplot(x='OnlineSecurity', hue='Churn', data=c_data, palette='Blues')
vc = c_data['OnlineSecurity'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between Online Security and Churn')
plt.show()

#### Relationship between Streaming Movies and Churn 

In [ ]:
sns.set_style('darkgrid')     
sns.countplot(x='StreamingMovies', hue='Churn', data=c_data, palette='Blues')
vc = c_data['StreamingMovies'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between Streaming Movies and Churn')
plt.show()

#### Relationship between Streaming TV and Churn          

In [ ]:
sns.set_style('darkgrid')     
sns.countplot(x='StreamingTV', hue='Churn', data=c_data, palette='Blues')
vc = c_data['StreamingTV'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between Streaming TV and Churn')
plt.show()

#### Relationship between Tech Support and Churn      

In [ ]:
sns.set_style('darkgrid')     
sns.countplot(x='TechSupport', hue='Churn', data=c_data, palette='Blues')
vc = c_data['TechSupport'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between TechSupport and Churn')
plt.show()

#### Relationship between Device Protection and Churn     

In [ ]:
sns.set_style('darkgrid')     
sns.countplot(x='DeviceProtection', hue='Churn', data=c_data, palette='Blues')
vc = c_data['DeviceProtection'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between Device Protection and Churn')
plt.show()

#### Relationship between Senior Citizen and Churn

In [ ]:
sns.set_style('darkgrid')     
sns.countplot(x='SeniorCitizen', hue='Churn', data=c_data, palette='Blues')
vc = c_data['SeniorCitizen'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between Senior Citizen and Churn')
plt.show()

-         


#### Relationship between Internet Service and Churn


In [ ]:
sns.set_style('darkgrid')
sns.countplot(x='InternetService', hue='Churn', data=c_data, palette='Blues')
vc = c_data['InternetService'].value_counts()
for p in ax.patches:
    # Get the x and y coordinates of the bar
    x_coord = p.get_x() + p.get_width() / 2
    y_coord = p.get_height()
    
    # Format the text you want to display as the annotation
    count = int(y_coord)  # Convert the count to an integer if needed
    text = f'{count}'  # Customize the text as desired
    
    # Add the annotation to the plot
    ax.text(x=x_coord, y=y_coord/2, s=text, ha='center', va='bottom')
plt.tight_layout()
plt.title('Relationship between Internet Service and Churn')
plt.show()


## Important Note

## Multvarite Analysis

In [ ]:
# Multvarite 
# correlation and heatmap
# pair plot     
#sns.pairplot(c_data, hue='Churn')
#sns.boxplot(x='Contract', y='MonthlyCharges', hue='Churn', data=c_data)


## Encoding of Categorical Columns

In [ ]:
LE = LabelEncoder()
c_data['gender'] = LE.fit_transform(c_data['gender'])
c_data['Partner'] = LE.fit_transform(c_data['Partner'])
c_data['Dependents'] = LE.fit_transform(c_data['Dependents'])
c_data['PhoneService'] = LE.fit_transform(c_data['PhoneService'])
c_data['MultipleLines'] = LE.fit_transform(c_data['MultipleLines'])
c_data['InternetService'] = LE.fit_transform(c_data['InternetService'])
c_data['OnlineSecurity'] = LE.fit_transform(c_data['OnlineSecurity'])
c_data['OnlineBackup'] = LE.fit_transform(c_data['OnlineBackup'])
c_data['DeviceProtection'] = LE.fit_transform(c_data['DeviceProtection'])
c_data['TechSupport'] = LE.fit_transform(c_data['TechSupport'])
c_data['StreamingTV'] = LE.fit_transform(c_data['StreamingTV'])
c_data['StreamingMovies'] = LE.fit_transform(c_data['StreamingMovies'])
c_data['Contract'] = LE.fit_transform(c_data['Contract'])
c_data['PaperlessBilling'] = LE.fit_transform(c_data['PaperlessBilling'])
c_data['PaymentMethod'] = LE.fit_transform(c_data['PaymentMethod'])

In [ ]:
c_data.info()

In [ ]:
# Compute correlation matrix
correlation_matrix = c_data.corr()

# Plot correlation matrix as a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation of Variables with Churn')
plt.show()

In [ ]:
#c_data.drop('customerID', axis=1, inplace=True)
# save to csv

## Important Note

In [ ]:
#For machine learning use bagging and boosting, these do not need dealing with imbalance data. 
#Ensemble methods: Ensemble methods, such as bagging or boosting, can be effective in handling imbalanced datasets. 
#Techniques like Random Forests or Gradient Boosting can better handle class imbalance by combining multiple weak models.


#For other machine learning deal with imbalance data by using SMOTE (Synthetic Minority Over-sampling Technique), or ADASYN (Adaptive Synthetic Sampling).
#

In [ ]:
y= c_data['Churn']                         # Target Variable
X = c_data.drop('Churn', axis =1)          # Independent Variable

In [ ]:
scaler = StandardScaler()
cs = scaler.fit_transform(X)
csn = pd.DataFrame(cs, columns = X.columns)
csn.head(5)

In [ ]:
def plot_metric(confusion, name):
    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ["{0:0.0f}".format(value) for value in
    confusion.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
    confusion.flatten()/np.sum(confusion)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
    zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    ax = sns.heatmap(confusion, annot=labels, fmt='', cmap='viridis')
    ax.set_title(f'{name}\n\n');
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values ');
    ## Ticket labels - List must be in alphabetical order
    ax.xaxis.set_ticklabels(['False','True'])
    ax.yaxis.set_ticklabels(['False','True'])
    ## Display the visualization of the Confusion Matrix.
    plt.show()

## Machine Models With Resampling

In [ ]:
X_resampled,y_resampled = ADASYN().fit_resample(X,y)
X_resampled.shape,y_resampled.shape

In [ ]:
print('Original dataset shape {}'.format(Counter(y)))
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

In [ ]:
X_s = X_resampled
y_s = y_resampled

In [ ]:
# split data into train+validation set and test set
X_trainval, X_test, y_trainval, y_test = train_test_split(X_s, y_s, random_state=0)

# split train+validation set into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_trainval, y_trainval, random_state=1)

print("Size of training set: {}\n size of validation set: {}\n size of test set:" " {}\n".format(X_train.shape[0], X_valid.shape[0], X_test.shape[0]))


### Logistic Regression

In [ ]:
LPlr = LogisticRegression()

# Fit 'LPknn' to the training set
LPlr.fit(X_train, y_train)

# Predict Output
ly_predict = LPlr.predict(X_valid)
pred_LPlr = LPlr.predict_proba(X_valid)[:,1]

#Train and Test Scores
LPlr_Tr_Score = round(LPlr.score(X_train, y_train)*100, 2)
LPlr_Tt_Score = round(LPlr.score(X_valid, y_valid)*100, 2)
print('train set score: {:.2f}'.format(LPlr_Tr_Score))
print('test set score: {:.2f}'.format(LPlr_Tt_Score))


print()
LPlr_sc = round(accuracy_score(y_valid, ly_predict)*100, 2)
print("Accuracy Score: {}%".format(LPlr_sc))
print()
PS_lr = round(precision_score(y_valid, ly_predict)*100, 2)
print("Precision Score: {}%".format(PS_lr))
print()
RS_lr = round(recall_score(y_valid, ly_predict)*100, 2)
print("Recall Score: {}%".format(RS_lr))
print()
FS_lr = round(f1_score(y_valid, ly_predict)*100, 2)
print("F1 Score: {}%".format(FS_lr))
print()
fpr1, tpr1, threshold = roc_curve(y_valid, pred_LPlr)
roc_auc = metrics.auc(fpr1, tpr1)
LPlrA = round(metrics.auc(fpr1, tpr1)*100, 2)
print(f'AUC score is:', LPlrA,'%')
print()
confusion = confusion_matrix(y_valid, ly_predict)
#print("Confusion matrix:\n{}".format(confusion))
plot_metric(confusion, "Logistic Regression")

#### **Confusion Matrix**
- **Logistic Regression Classifier** classified **740** about **38%**, correctly  **[True Negative]**, which is the correct prediction of customers who did not churn and failed to classify **245** about **13%**, correctly **[False Positive]**, which is the incorrect prediction of customers who did not churn.

- **Logistic Regression Classifier** classified **786** about **40%**, correctly  **[True Positive]**, which is the correct prediction of customers who churn and failed to classify **178** about **9%**, correctly **[False Negative]**, which is the incorrect prediction of customeers who did churn.

### RandomForest

In [ ]:
crf = RandomForestClassifier()
# Fit 'LPknn' to the training set
crf.fit(X_train, y_train)

# Predict Output
crf_predict = crf.predict(X_valid)
pred_crf = crf.predict_proba(X_valid)[:,1]

#Train and Test Scores
crf_Tr_Score = round(crf.score(X_train, y_train)*100, 2)
crf_Tt_Score = round(crf.score(X_valid, y_valid)*100, 2)
print('train set score: {:.2f}'.format(crf_Tr_Score))
print('test set score: {:.2f}'.format(crf_Tt_Score))



print()
crf_sc = round(accuracy_score(y_valid, crf_predict)*100, 2)
print("Accuracy Score: {}%".format(crf_sc))
print()
PS_crf = round(precision_score(y_valid, crf_predict)*100, 2)
print("Precision Score: {}%".format(PS_crf))
print()
RS_crf = round(recall_score(y_valid, crf_predict)*100, 2)
print("Recall Score: {}%".format(RS_crf))
print()
FS_crf = round(f1_score(y_valid, crf_predict)*100, 2)
print("F1 Score: {}%".format(FS_crf))
print()
fpr2, tpr2, threshold = roc_curve(y_valid, pred_crf)
roc_auc = metrics.auc(fpr2, tpr2)
crfA = round(metrics.auc(fpr2, tpr2)*100, 2)
print(f'AUC score is:', crfA,'%')
print()
confusion = confusion_matrix(y_valid, crf_predict)
#print("Confusion matrix:\n{}".format(confusion))
plot_metric(confusion, "Random Forest")

#### **Confusion Matrix**
- **Random Forest Classifier** classified **827** about **42%**, correctly  **[True Negative]**, which is the correct prediction of customers who did not churn and failed to classify **158** about **8%**, correctly **[False Positive]**, which is the incorrect prediction of customers who did not churn.

- **Random Forest Classifier** classified **801** about **41%**, correctly  **[True Positive]**, which is the correct prediction of customers who churn and failed to classify **163** about **8%**, correctly **[False Negative]**, which is the incorrect prediction of customeers who did churn.

### Support Vector Machine


In [ ]:
csvc = SVC(gamma='auto', probability=True)
# Fit 'LPknn' to the training set
csvc.fit(X_train, y_train)

# Predict Output
csvc_predict = csvc.predict(X_valid)
pred_csvc = csvc.predict_proba(X_valid)[:,1]
#pred_crf = crf.predict_proba(X_valid)[:,1]

#Train and Test Scores
csvc_Tr_Score = round(csvc.score(X_train, y_train)*100, 2)
csvc_Tt_Score = round(csvc.score(X_valid, y_valid)*100, 2)
print('train set score: {:.2f}'.format(csvc_Tr_Score))
print('test set score: {:.2f}'.format(csvc_Tt_Score))



print()
csvc_sc = round(accuracy_score(y_valid, csvc_predict)*100, 2)
print("Accuracy Score: {}%".format(csvc_sc))
print()
PS_csvc = round(precision_score(y_valid, csvc_predict)*100, 2)
print("Precision Score: {}%".format(PS_csvc))
print()
RS_csvc = round(recall_score(y_valid, csvc_predict)*100, 2)
print("Recall Score: {}%".format(RS_csvc))
print()
FS_csvc = round(f1_score(y_valid, csvc_predict)*100, 2)
print("F1 Score: {}%".format(FS_csvc))
print()
fpr3, tpr3, threshold = roc_curve(y_valid, pred_csvc)
roc_auc = metrics.auc(fpr3, tpr3)
csvcA = round(metrics.auc(fpr3, tpr3)*100, 2)
print(f'AUC score is:', csvcA,'%')
print()
confusion = confusion_matrix(y_valid, csvc_predict)
#print("Confusion matrix:\n{}".format(confusion))
plot_metric(confusion, "Support Vector Machine")

#### **Confusion Matrix**
- **Support Vector Machine Classifier** classified **810** about **42%**, correctly  **[True Negative]**, which is the correct prediction of customers who did not churn and failed to classify **175** about **9%**, correctly **[False Positive]**, which is the incorrect prediction of customers who did not churn.

- **Support Vector Machine Classifier** classified **709** about **36%**, correctly  **[True Positive]**, which is the correct prediction of customers who churn and failed to classify **255** about **13%**, correctly **[False Negative]**, which is the incorrect prediction of customeers who did churn.

### K- Nearest Neighbors

In [ ]:
cknn = KNeighborsClassifier(n_neighbors=7)

# Fit 'LPknn' to the training set
cknn.fit(X_train, y_train)

# Predict Output
cknn_predict = cknn.predict(X_valid)
pred_cknn = cknn.predict_proba(X_valid)[:,1]

#Train and Test Scores
cknn_Tr_Score = round(cknn.score(X_train, y_train)*100, 2)
cknn_Tt_Score = round(cknn.score(X_valid, y_valid)*100, 2)
print('train set score: {:.2f}'.format(cknn_Tr_Score))
print('test set score: {:.2f}'.format(cknn_Tt_Score))



print()
cknn_sc = round(accuracy_score(y_valid, cknn_predict)*100, 2)
print("Accuracy Score: {}%".format(cknn_sc))
print()
PS_cknn = round(precision_score(y_valid, cknn_predict)*100, 2)
print("Precision Score: {}%".format(PS_cknn))
print()
RS_cknn = round(recall_score(y_valid, cknn_predict)*100, 2)
print("Recall Score: {}%".format(RS_cknn))
print()
FS_cknn = round(f1_score(y_valid, cknn_predict)*100, 2)
print("F1 Score: {}%".format(FS_cknn))
print()
fpr4, tpr4, threshold = roc_curve(y_valid, pred_cknn)
roc_auc = metrics.auc(fpr4, tpr4)
cknnA = round(metrics.auc(fpr4, tpr4)*100, 2)
print(f'AUC score is:', cknnA,'%')
print()
confusion = confusion_matrix(y_valid, cknn_predict)
#print("Confusion matrix:\n{}".format(confusion))
plot_metric(confusion, "K- Nearest Neighbors")

#### **Confusion Matrix**
- **K- Nearest Neighbors Classifier** classified **629** about **32%**, correctly  **[True Negative]**, which is the correct prediction of customers who did not churn and failed to classify **356** about **18%**, correctly **[False Positive]**, which is the incorrect prediction of customers who did not churn.

- **K- Nearest Neighbors Classifier** classified **789** about **40%**, correctly  **[True Positive]**, which is the correct prediction of customers who churn and failed to classify **175** about **9%**, correctly **[False Negative]**, which is the incorrect prediction of customeers who did churn.

### Extra Trees Classifier

In [ ]:
crc = ExtraTreesClassifier()

# Fit 'LPknn' to the training set
crc.fit(X_train, y_train)

# Predict Output
crc_predict = crc.predict(X_valid)
pred_crc = crc.predict_proba(X_valid)[:,1]

#Train and Test Scores
crc_Tr_Score = round(crc.score(X_train, y_train)*100, 2)
crc_Tt_Score = round(crc.score(X_valid, y_valid)*100, 2)
print('train set score: {:.2f}'.format(crc_Tr_Score))
print('test set score: {:.2f}'.format(crc_Tt_Score))



print()
crc_sc = round(accuracy_score(y_valid, crc_predict)*100, 2)
print("Accuracy Score: {}%".format(crc_sc))
print()
PS_crc = round(precision_score(y_valid, crc_predict)*100, 2)
print("Precision Score: {}%".format(PS_crc))
print()
RS_crc = round(recall_score(y_valid, crc_predict)*100, 2)
print("Recall Score: {}%".format(RS_crc))
print()
FS_crc = round(f1_score(y_valid, crc_predict)*100, 2)
print("F1 Score: {}%".format(FS_crc))
print()
fpr5, tpr5, threshold = roc_curve(y_valid, pred_crc)
roc_auc = metrics.auc(fpr5, tpr5)
crcA = round(metrics.auc(fpr5, tpr5)*100, 2)
print(f'AUC score is:', crcA,'%')
print()
confusion = confusion_matrix(y_valid, crc_predict)
#print("Confusion matrix:\n{}".format(confusion))
plot_metric(confusion, "Extra Trees Classifier")

#### **Confusion Matrix**
- **Extra Trees Classifier** classified **826** about **42%**, correctly  **[True Negative]**, which is the correct prediction of customers who did not churn and failed to classify **159** about **8%**, correctly **[False Positive]**, which is the incorrect prediction of customers who did not churn.

- **Extra Trees Classifier** classified **783** about **40%**, correctly  **[True Positive]**, which is the correct prediction of customers who churn and failed to classify **181** about **9%**, correctly **[False Negative]**, which is the incorrect prediction of customeers who did churn.

### Histogram-based Gradient Boosting Classification Tree 



In [ ]:
LPdt = HistGradientBoostingClassifier() 

#Fit 'LPdt' to the training set
LPdt.fit(X_train, y_train)

# Predict Output
dy_predict = LPdt.predict(X_valid)
pred_LPdt = LPdt.predict_proba(X_valid)[:,1]

#Train and Test Scores
LPdt_Tr_Score = round(LPdt.score(X_train, y_train)*100, 2)
LPdt_Tt_Score = round(LPdt.score(X_valid, y_valid)*100, 2)
print('train set score: {:.2f}'.format(LPdt_Tr_Score))
print('test set score: {:.2f}'.format(LPdt_Tt_Score))


print()
LPdt_sc = round(accuracy_score(y_valid, dy_predict)*100, 2)
print("Accuracy Score: {}%".format(LPdt_sc))
print()
PS_dt = round(precision_score(y_valid, dy_predict)*100, 2)
print("Precision Score: {}%".format(PS_dt))
print()
RS_dt = round(recall_score(y_valid, dy_predict)*100, 2)
print("Recall Score: {}%".format(RS_dt))
print()
FS_dt = round(f1_score(y_valid, dy_predict)*100, 2)
print("F1 Score: {}%".format(FS_dt))
print()
fpr6, tpr6, threshold = roc_curve(y_valid, pred_LPdt)
roc_auc = metrics.auc(fpr6, tpr6)
LPhgbA = round(metrics.auc(fpr6, tpr6)*100, 2)
print(f'AUC score is:', LPhgbA,'%')
print()
confusion = confusion_matrix(y_valid, dy_predict)
#print("Confusion matrix:\n{}".format(confusion))
print()
plot_metric(confusion, "HistGradientBoosting")

#### **Confusion Matrix**
- **Histogram-based Gradient Boosting Classifier** classified **814** about **42%**, correctly  **[True Negative]**, which is the correct prediction of customers who did not churn and failed to classify **171** about **9%**, correctly **[False Positive]**, which is the incorrect prediction of customers who did not churn.

- **Histogram-based Gradient Boosting Classifier** classified **801** about **41%**, correctly  **[True Positive]**, which is the correct prediction of customers who churn and failed to classify **163** about **8%**, correctly **[False Negative]**, which is the incorrect prediction of customeers who did churn.

### AdaBoostClassifier

In [ ]:
LPrf = AdaBoostClassifier(n_estimators=100, random_state=0)

#Fit 'LPrf' to the training set
LPrf.fit(X_train, y_train)

# Predict Output
ry_predict = LPrf.predict(X_valid)
pred_LPrf = LPrf.predict_proba(X_valid)[:,1]

#Train and Test Scores
LPrf_Tr_Score = round(LPrf.score(X_train, y_train)*100, 2)
LPrf_Tt_Score = round(LPrf.score(X_valid, y_valid)*100, 2)
print('train set score: {:.2f}'.format(LPrf_Tr_Score))
print('test set score: {:.2f}'.format(LPrf_Tt_Score))


print()
LPrf_sc = round(accuracy_score(y_valid, ry_predict)*100, 2)
print("Accuracy Score: {}%".format(LPrf_sc))
print()
PS_rf = round(precision_score(y_valid, ry_predict)*100, 2)
print("Precision Score: {}%".format(PS_rf))
print()
RS_rf = round(recall_score(y_valid, ry_predict)*100, 2)
print("Recall Score: {}%".format(RS_rf))
print()
FS_rf = round(f1_score(y_valid, ry_predict)*100, 2)
print("F1 Score: {}%".format(FS_rf))
print()
fpr7, tpr7, threshold = roc_curve(y_valid, pred_LPrf)
roc_auc = metrics.auc(fpr7, tpr7)
LPabtA = round(metrics.auc(fpr7, tpr7)*100, 2)
print(f'AUC score is:', LPabtA,'%')
print()
confusion = confusion_matrix(y_valid, ry_predict)
#print("Confusion matrix:\n{}".format(confusion))
print()
plot_metric(confusion, "AdaBoost")

#### **Confusion Matrix**
- **AdaBoost Classifier** classified **765** about **39%**, correctly  **[True Negative]**, which is the correct prediction of customers who did not churn and failed to classify **220** about **11%**, correctly **[False Positive]**, which is the incorrect prediction of customers who did not churn.

- **AdaBoost Classifier** classified **831** about **43%**, correctly  **[True Positive]**, which is the correct prediction of customers who churn and failed to classify **133** about **7%**, correctly **[False Negative]**, which is the incorrect prediction of customeers who did churn.

### GradientBoostingClassifier


In [ ]:
LPgbm = GradientBoostingClassifier()

#Fit 'LPsvm' to the training set
LPgbm.fit(X_train, y_train)

# Predict Output
gy_predict = LPgbm.predict(X_valid)
pred_LPgbm = LPgbm.predict_proba(X_valid)[:,1]


#Train and Test Scores
LPgbm_Tr_Score = round(LPgbm.score(X_train, y_train)*100, 2)
LPgbm_Tt_Score = round(LPgbm.score(X_valid, y_valid)*100, 2)
print('train set score: {:.2f}'.format(LPrf_Tr_Score))
print('test set score: {:.2f}'.format(LPgbm_Tt_Score))


print()
LPgbm_sc = round(accuracy_score(y_valid, gy_predict)*100, 2)
print("Accuracy Score: {}%".format(LPgbm_sc))
print()
PS_gbm = round(precision_score(y_valid, gy_predict)*100, 2)
print("Precision Score: {}%".format(PS_gbm))
print()
RS_gbm = round(recall_score(y_valid, gy_predict)*100, 2)
print("Recall Score: {}%".format(RS_gbm))
print()
FS_gbm = round(f1_score(y_valid, gy_predict)*100, 2)
print("F1 Score: {}%".format(FS_gbm))
print()
fpr8, tpr8, threshold = roc_curve(y_valid, pred_LPgbm)
roc_auc = metrics.auc(fpr8, tpr8)
LPgbmA = round(metrics.auc(fpr8, tpr8)*100, 2)
print(f'AUC score is:', LPgbmA,'%')
print()
confusion = confusion_matrix(y_valid, gy_predict)
#print("Confusion matrix:\n{}".format(confusion))
plot_metric(confusion, "Gradient Boosting")

#### **Confusion Matrix**
- **Gradient Boosting Classifier** classified **785** about **40%**, correctly  **[True Negative]**, which is the correct prediction of customers who did not churn and failed to classify **200** about **10%**, correctly **[False Positive]**, which is the incorrect prediction of customers who did not churn.

- **Gradient Boosting Classifier** classified **821** about **42%**, correctly  **[True Positive]**, which is the correct prediction of customers who churn and failed to classify **143** about **7%**, correctly **[False Negative]**, which is the incorrect prediction of customeers who did churn.


## Find the best Model

### Evaluation Metric Summary

In [ ]:
LPMetric = pd.DataFrame({'Models': ["Logistic Regression",
                                    "RandomForest",
                                    "Support Vector Machine",
                                    "K- Nearest Neighbors",
                                    "Extra Trees Classifier",  
                                    "HistGradientBoosting",
                                    "AdaBoost",
                                    "Gradient Boosting"],
                             'Accuracy Score': [LPlr_sc, crf_sc, csvc_sc, cknn_sc, crc_sc, LPdt_sc, LPrf_sc, LPgbm_sc],
                             'Precision Score': [PS_lr, PS_crf, PS_csvc, PS_cknn, PS_crc, PS_dt, PS_rf, PS_gbm],
                             'Recall Score': [RS_lr, RS_crf, RS_csvc, RS_cknn, RS_crc, RS_dt, RS_rf, RS_gbm],
                             'F1 Score': [FS_lr, FS_crf, FS_csvc, FS_cknn, FS_crc, FS_dt, FS_rf, FS_gbm]})


LPMetrics = LPMetric.sort_values(by = 'Accuracy Score',ascending = False)
LPMetrics = LPMetrics.set_index('Models')
LPMetrics

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(fpr1,tpr1, label = 'Logistic Regression ' 'AUC = %0.2f' %  LPlrA)
plt.plot(fpr2,tpr2, label = 'RandomForest ' 'AUC = %0.2f' % crfA)
plt.plot(fpr3,tpr3, label = 'Support Vector Machine ' 'AUC = %0.2f' % csvcA)
plt.plot(fpr4,tpr4, label = 'K- Nearest Neighbors ' 'AUC = %0.2f' % cknnA)
plt.plot(fpr5,tpr5, label = 'Extra Trees Classifier ' 'AUC = %0.2f' % crcA)
plt.plot(fpr6,tpr6, label ='HistGradientBoosting ' 'AUC = %0.2f' % LPhgbA)
plt.plot(fpr7,tpr7, label = 'AdaBoost Classifier ' 'AUC = %0.2f' % LPabtA)
plt.plot(fpr8,tpr8, label = 'Gradient Boosting ' 'AUC = %0.2f' % LPgbmA)
plt.legend(loc = 'best')
plt.xlabel('FPR', fontsize = 15)
plt.ylabel('TPR', fontsize = 15)
plt.title('RECIEVER OPERATING CHARACTERISTICS', fontsize = 20)
plt.show()

### **Feature Importance**

In [ ]:
importance = pd.DataFrame({'feature':X.columns,'importance':np.round(crf.feature_importances_,3)})
importance = importance.sort_values('importance',ascending=False).set_index('feature')
importance.head(30)

In [ ]:
importance.plot.barh()

#### **Dropping insignificant features** 

In [ ]:
X.drop('PhoneService', axis = 1, inplace = True)
X.drop('SeniorCitizen', axis = 1, inplace = True)
X.drop('StreamingMovies', axis = 1, inplace = True)
X.drop('StreamingTV', axis = 1, inplace = True)
X.head()

In [ ]:
#from sklearn.ensemble import 
model = RandomForestClassifier()


# define grid search
grid = {
    'n_estimators': [100, 200, 300],  # try different number of trees
    'max_depth': [None, 5, 10],       # try different maximum depth values
    #'min_samples_split': [2, 5, 10],  # try different values for minimum samples required to split
    #'min_samples_leaf': [1, 2, 4],    # try different values for minimum samples required at a leaf node
    'max_features': ['sqrt', 'log2'], # try different options for the number of features to consider
    #'criterion': ['gini', 'entropy']  # try different quality measures
}
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

LPdt = RandomForestClassifier(**grid_result.best_params_, max_bins=255, max_iter=100)

#Fit 'LPdt' to the training set
LPdt.fit(X_trainval, y_trainval)

# Predict Output
dy_predict = LPdt.predict(X_test)
pred_LPdt = LPdt.predict_proba(X_test)[:,1]

#Train and Test Scores
LPdt_Tr_Score = round(LPdt.score(X_trainval, y_trainval)*100, 2)
LPdt_Tt_Score = round(LPdt.score(X_test, y_test)*100, 2)
print('train set score: {:.2f}'.format(LPdt_Tr_Score))
print('test set score: {:.2f}'.format(LPdt_Tt_Score))